# Web Scraping with Beautiful Soup

* * * 

### Iconos utilizados en este notebook
🔔 **Pregunta**: Una pregunta rápida para ayudarte a entender lo que está pasando.<br>
🥊 **Desafío**: Ejercicio interactivo. Los trabajaremos en el taller.<br>
⚠️ **Advertencia**: Aviso sobre cosas complicadas o errores comunes.<br>
💡 **Consejo**: Cómo hacer algo un poco más eficiente o eficaz.<br>
🎬 **Demo**: Mostrar algo más avanzado - ¡Para que sepas para qué se puede usar Python!<br>

### Objetivos de aprendizaje
1. [Reflexión: Extraer o no extraer](#when)
2. [Extracción y análisis de HTML](#extract)
3. [Raspado de la Asamblea General de Illinois](#scrape)

<a id='when'></a>

# Extraer o no extraer

Cuando queremos acceder a datos de la web, primero tenemos que asegurarnos de si el sitio web que nos interesa ofrece una API web. Plataformas como Twitter, Reddit y el New York Times ofrecen APIs. **Consulta el taller [Python Web APIs](https://github.com/dlab-berkeley/Python-Web-APIs) de D-Lab si quieres aprender a utilizar APIs.**

Sin embargo, a menudo hay casos en los que no existe una API Web. En estos casos, puede que tengamos que recurrir al web scraping, donde extraemos el HTML subyacente de una página web, y obtenemos directamente la información que queremos. Hay varios paquetes en Python que podemos utilizar para realizar estas tareas. Nos centraremos en dos paquetes: Requests y Beautiful Soup.

Nuestro caso de estudio será obtener información sobre los [senadores estatales de Illinois](http://www.ilga.gov/senate), así como la [lista de proyectos de ley](http://www.ilga.gov/senate/SenatorBills.asp?MemberID=1911&GA=98&Primary=True) que cada senador ha patrocinado. Antes de empezar, echa un vistazo a estos sitios web para ver su estructura.

## Instalación

Utilizaremos dos paquetes principales: [Requests](http://docs.python-requests.org/en/latest/user/quickstart/) and [Beautiful Soup](http://www.crummy.com/software/BeautifulSoup/bs4/doc/). Instala estos paquetes, si aún no lo has hecho:

In [17]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [19]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


También instalaremos el paquete `lxml`, que ayuda a soportar parte del análisis sintáctico que realiza Beautiful Soup:

In [23]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [28]:
# Importar las bibliotecas necesarias
from bs4 import BeautifulSoup
from datetime import datetime
import requests
import time

<a id='extract'></a>

# Extracción y análisis de HTML 

Para extraer y analizar HTML con éxito, seguiremos los 4 pasos siguientes:
1. Realizar una petición GET
2. Parsear la página con Beautiful Soup
3. Buscar elementos HTML
4. Obtener atributos y texto de estos elementos

## Paso 1: Realizar una petición GET para obtener el HTML de una página

Podemos utilizar la biblioteca Requests para:

1. Hacer una petición GET a la página, y
2. Leer el código HTML de la página web.

El proceso de hacer una petición y obtener un resultado se parece al del flujo de trabajo de la Web API. Ahora, sin embargo, estamos haciendo una solicitud directamente a la página web, y vamos a tener que analizar el código HTML nosotros mismos. Esto contrasta con los datos que se nos proporcionan organizados en una salida `JSON` o `XML` más directa.

In [33]:
# Realizar una solicitud GET
req = requests.get('http://www.ilga.gov/senate/default.asp')
# Leer el contenido de la respuesta del servidor
src = req.text
# Ver algunos resultados
print(src[:1000])

<html lang="en"> 
<!-- Trigger/Open The Modal -->
<div style="position: fixed; z-index: 999; top: 5; left: 600; background-color: navy; display: block">
<button id="myBtn" style="color: white; background-color: navy; display: block">Translate Website</button></div>
<!-- The Modal -->
<div id="myModal" class="modal" style="display: none">
  <!-- Modal content -->
  <div class="modal-content">
      <div class="modal-header"><h3>
    <span class="close">&times;</span></h3></div>    
    <p>The Illinois General Assembly offers the Google Translate service for visitor convenience. In no way should it be considered accurate as to the translation of any content herein.</p>
    <p>Visitors of the Illinois General Assembly website are encouraged to use other translation services available on the internet.</p>
    <p>The English language version is always the official and authoritative version of this website.</p>
    <p>NOTE: To return to the original English language version, se


## Paso 2: Analizar la página con Beautiful Soup

Ahora, usamos la función `BeautifulSoup` para parsear la respuesta en un árbol HTML. Esto devuelve un objeto (llamado **objeto sopa**) que contiene todo el HTML del documento original.

Si te encuentras con un error sobre una biblioteca de análisis, asegúrate de que has instalado el paquete `lxml` para proporcionar a Beautiful Soup las herramientas de análisis necesarias.

In [54]:
# Parsear la respuesta en un árbol HTML
soup = BeautifulSoup(src, 'lxml')
# Muestra resultado
print(soup.prettify()[:1000])

<html lang="en">
 <!-- Trigger/Open The Modal -->
 <body>
  <div style="position: fixed; z-index: 999; top: 5; left: 600; background-color: navy; display: block">
   <button id="myBtn" style="color: white; background-color: navy; display: block">
    Translate Website
   </button>
  </div>
  <!-- The Modal -->
  <div class="modal" id="myModal" style="display: none">
   <!-- Modal content -->
   <div class="modal-content">
    <div class="modal-header">
     <h3>
      <span class="close">
       ×
      </span>
     </h3>
    </div>
    <p>
     The Illinois General Assembly offers the Google Translate service for visitor convenience. In no way should it be considered accurate as to the translation of any content herein.
    </p>
    <p>
     Visitors of the Illinois General Assembly website are encouraged to use other translation services available on the internet.
    </p>
    <p>
     The English language version is always the official and authoritative version of this website.
   

La salida es bastante similar a la anterior, pero ahora está organizada en un objeto `soup` que nos permite recorrer la página más fácilmente.

## Paso 3: Buscar elementos HTML

Beautiful Soup tiene una serie de funciones para encontrar componentes útiles en una página. Beautiful Soup te permite encontrar elementos por su:

1. Etiquetas HTML
2. Atributos HTML
3. Selectores CSS

Busquemos primero **etiquetas HTML**. 

La función `find_all` busca en el árbol `soup` para encontrar todos los elementos con una etiqueta HTML en particular, y devuelve todos esos elementos.

¿Qué hace el siguiente ejemplo?

In [41]:
# Buscar todos los elementos con una determinada etiqueta
a_tags = soup.find_all("a")
print(a_tags[:10])

[<a class="goog-logo-link" href="https://translate.google.com" target="_blank"><img alt="Google Translate" height="14" src="https://www.gstatic.com/images/branding/googlelogo/1x/googlelogo_color_42x16dp.png" style="padding-right: 3px;" width="37"/>Translate</a>, <a href="/default.asp"><img alt="Illinois General Assembly" border="0" height="49" src="/images/logo_sm.gif" width="462"/></a>, <a class="mainmenu" href="/">Home</a>, <a class="mainmenu" href="/legislation/" onblur="HM_f_PopDown('elMenu1')" onfocus="HM_f_PopUp('elMenu1',event)" onmouseout="HM_f_PopDown('elMenu1')" onmouseover="HM_f_PopUp('elMenu1',event)">Legislation &amp; Laws</a>, <a class="mainmenu" href="/senate/" onblur="HM_f_PopDown('elMenu3')" onfocus="HM_f_PopUp('elMenu3',event)" onmouseout="HM_f_PopDown('elMenu3')" onmouseover="HM_f_PopUp('elMenu3',event)">Senate</a>, <a class="mainmenu" href="/house/" onblur="HM_f_PopDown('elMenu2')" onfocus="HM_f_PopUp('elMenu2',event)" onmouseout="HM_f_PopDown('elMenu2')" onmouseove

Como `find_all()` es el método más popular de la API de búsqueda de Beautiful Soup, puedes usar un atajo para él. Si tratas el objeto BeautifulSoup como si fuera una función, entonces es lo mismo que llamar a `find_all()` en ese objeto. 

Estas dos líneas de código son equivalentes:

In [44]:
a_tags = soup.find_all("a")
a_tags_alt = soup("a")
print(a_tags[0])
print(a_tags_alt[0])

<a class="goog-logo-link" href="https://translate.google.com" target="_blank"><img alt="Google Translate" height="14" src="https://www.gstatic.com/images/branding/googlelogo/1x/googlelogo_color_42x16dp.png" style="padding-right: 3px;" width="37"/>Translate</a>
<a class="goog-logo-link" href="https://translate.google.com" target="_blank"><img alt="Google Translate" height="14" src="https://www.gstatic.com/images/branding/googlelogo/1x/googlelogo_color_42x16dp.png" style="padding-right: 3px;" width="37"/>Translate</a>


¿Cuántos enlaces hemos obtenido?

In [46]:
print(len(a_tags))

213


Son muchos. Muchos elementos de una página tendrán la misma etiqueta HTML. Por ejemplo, si buscas todo lo que tenga la etiqueta `a`, es probable que obtengas más resultados, muchos de los cuales puede que no te interesen. Recuerda que la etiqueta `a` define un hipervínculo, por lo que normalmente encontrarás muchos en cualquier página.

¿Y si quisiéramos buscar etiquetas HTML con determinados atributos, como clases CSS concretas? 

Podemos hacerlo añadiendo un argumento adicional a `find_all`. En el ejemplo siguiente, buscamos todas las etiquetas `a` y filtramos las que tienen `class_=«sidemenu»`.

In [49]:
# Obtener sólo las etiquetas 'a' en la clase 'sidemenu
side_menus = soup("a", class_="sidemenu")
side_menus[:5]

[<a class="sidemenu" href="/senate/default.asp">  Members  </a>,
 <a class="sidemenu" href="/senate/committees/default.asp">  Committees  </a>,
 <a class="sidemenu" href="/senate/schedules/default.asp">  Schedules  </a>,
 <a class="sidemenu" href="/senate/journals/default.asp">  Journals  </a>,
 <a class="sidemenu" href="/senate/transcripts/default.asp">  Transcripts  </a>]

Una forma más eficiente de buscar elementos en una web es mediante un **selector CSS**. Para ello tenemos que utilizar un método diferente llamado `select()`. Basta con pasar una cadena en `.select()` para obtener todos los elementos con esa cadena como selector CSS válido.

En el ejemplo anterior, podemos utilizar `«a.sidemenu»` como un selector CSS, que devuelve todas las etiquetas `a` con la clase `sidemenu`.

In [52]:
# Obtener elementos con selector CSS «a.sidemenu».
selected = soup.select("a.sidemenu")
selected[:5]

[<a class="sidemenu" href="/senate/default.asp">  Members  </a>,
 <a class="sidemenu" href="/senate/committees/default.asp">  Committees  </a>,
 <a class="sidemenu" href="/senate/schedules/default.asp">  Schedules  </a>,
 <a class="sidemenu" href="/senate/journals/default.asp">  Journals  </a>,
 <a class="sidemenu" href="/senate/transcripts/default.asp">  Transcripts  </a>]

## 🥊 Desafío: Encontrar todo

Utiliza BeautifulSoup para encontrar todos los elementos `a` con clase `mainmenu`.

In [58]:
# Obtener elementos con selector CSS «a.mainmenu».
soup.select("a.mainmenu")

[<a class="mainmenu" href="/">Home</a>,
 <a class="mainmenu" href="/legislation/" onblur="HM_f_PopDown('elMenu1')" onfocus="HM_f_PopUp('elMenu1',event)" onmouseout="HM_f_PopDown('elMenu1')" onmouseover="HM_f_PopUp('elMenu1',event)">Legislation &amp; Laws</a>,
 <a class="mainmenu" href="/senate/" onblur="HM_f_PopDown('elMenu3')" onfocus="HM_f_PopUp('elMenu3',event)" onmouseout="HM_f_PopDown('elMenu3')" onmouseover="HM_f_PopUp('elMenu3',event)">Senate</a>,
 <a class="mainmenu" href="/house/" onblur="HM_f_PopDown('elMenu2')" onfocus="HM_f_PopUp('elMenu2',event)" onmouseout="HM_f_PopDown('elMenu2')" onmouseover="HM_f_PopUp('elMenu2',event)">House</a>,
 <a class="mainmenu" href="/mylegislation/" onblur="HM_f_PopDown('elMenu4')" onfocus="HM_f_PopUp('elMenu4',event)" onmouseout="HM_f_PopDown('elMenu4')" onmouseover="HM_f_PopUp('elMenu4',event)">My Legislation</a>,
 <a class="mainmenu" href="/sitemap.asp">Site Map</a>]

## Paso 4: Obtener atributos y texto de los elementos

Una vez identificados los elementos, queremos la información de acceso a ese elemento. Normalmente, esto significa dos cosas

1. Texto
2. Atributos

Obtener el texto dentro de un elemento es fácil. Todo lo que tenemos que hacer es utilizar el miembro `text` de un objeto `tag`:

In [61]:
# Obtener todos los enlaces sidemenu como una lista
side_menu_links = soup.select("a.sidemenu")

# Examinar el primer enlace
first_link = side_menu_links[0]
print(first_link)

# ¿Qué clase es esta variable?
print('Class: ', type(first_link))

<a class="sidemenu" href="/senate/default.asp">  Members  </a>
Class:  <class 'bs4.element.Tag'>


Es una etiqueta Beautiful Soup. Esto significa que tiene un miembro `text`:

In [64]:
print(first_link.text)

  Members  


A veces queremos el valor de ciertos atributos. Esto es particularmente relevante para las etiquetas `a`, o enlaces, donde el atributo `href` nos dice dónde va el enlace.

💡 **Consejo**: Puedes acceder a los atributos de una etiqueta tratando la etiqueta como un diccionario:

In [67]:
print(first_link['href'])

/senate/default.asp


## 🥊 Reto: Extraer atributos específicos

Extraer todos los atributos `href` de cada URL `mainmenu`.

In [69]:
[link['href'] for link in soup.select("a.mainmenu")]

['/',
 '/legislation/',
 '/senate/',
 '/house/',
 '/mylegislation/',
 '/sitemap.asp']

<a id='scrape'></a>

# Scraping de la Asamblea General de Illinois

Lo creas o no, esas son realmente las herramientas fundamentales que necesitas para hacer scraping de un sitio web. Una vez que pasas más tiempo familiarizándote con HTML y CSS, entonces es simplemente una cuestión de entender la estructura de un sitio web en particular y aplicar inteligentemente las herramientas de Beautiful Soup y Python.

Apliquemos estos conocimientos al scrapeo de la [98ª Asamblea General de Illinois](http://www.ilga.gov/senate/default.asp?GA=98).

En concreto, nuestro objetivo es obtener información sobre cada senador, incluyendo su nombre, distrito y partido.## Raspar y Parsear la Página Web

Vamos a raspar y analizar la página web, utilizando las herramientas que hemos aprendido en la sección anterior.

## Scrape and Soup the Webpage

Let's scrape and parse the webpage, using the tools we learned in the previous section.

In [73]:
# Realizar una petición GET
req = requests.get('http://www.ilga.gov/senate/default.asp?GA=98')
# Leer el contenido de la respuesta del servidor
src = req.text
# Ponerlo en formato soup
soup = BeautifulSoup(src, "lxml")

## Búsqueda de los elementos de la tabla

Nuestro objetivo es obtener los elementos de la tabla en la página web. Recuerda: las filas se identifican con la etiqueta `tr`. Usemos `find_all` para obtener estos elementos.

In [75]:
# Obtener todos los elementos de las filas de la tabla
rows = soup.find_all("tr")
len(rows)

73

⚠️ Advertencia: Tenga en cuenta: find_all obtiene todos los elementos con la etiqueta tr. Nosotros sólo queremos algunos de ellos. Si usamos la función 'Inspeccionar' de Google Chrome y miramos con atención, podemos usar algunos selectores CSS para obtener sólo las filas que nos interesan. En concreto, queremos las filas interiores de la tabla:

In [77]:
# Devuelve todos los selectores css 'tr tr tr' de la página
rows = soup.select('tr tr tr')

for row in rows[:5]:
    print(row, '\n')

<tr><td colspan="5">
<span class="heading">Illinois State Senators</span>
<span class="italics">  98th  General Assembly</span><br/>
<!-- 3/2/09 temp comment out until fixed for GA specific-->
<!-- add 97th ga currently no info -->
<a href="98GA_Senate_Leadership.pdf">Leadership</a> <a href="98th_Senate_Officers.pdf">Officers</a> <a href="98GA_Senate_Seating_Chart.pdf">Senate Seating Chart</a>  <span class="content"><b>Democrats:</b> 40   <b>Republicans:</b> 19</span><br/>
</td></tr> 

<tr>
<td class="header" width="45%"><a class="filetab" href="javascript:Sort('LastName','',98);" title="Sort by Senator">Senator</a></td>
<td align="center" class="header" width="15%">Bills</td>
<td align="center" class="header" width="10%">Committees</td>
<td align="center" class="header" width="15%"><a class="filetab" href="javascript:Sort('DistrictNumber','',98);" title="Sort by District">District</a></td>
<td align="center" class="header" width="15%"><a class="filetab" href="javascript:Sort('Party','

Parece que queremos todo después de las dos primeras filas. Vamos a trabajar con una sola fila para empezar, y construir nuestro bucle a partir de ahí.

In [80]:
example_row = rows[2]
print(example_row.prettify())

<tr>
 <td bgcolor="white" class="detail" width="40%">
  <a class="notranslate" href="/senate/Senator.asp?GA=98&amp;MemberID=1911">
   Pamela J. Althoff
  </a>
 </td>
 <td align="center" bgcolor="white" class="detail" width="15%">
  <a href="SenatorBills.asp?GA=98&amp;MemberID=1911">
   Bills
  </a>
 </td>
 <td align="center" bgcolor="white" class="detail" width="15%">
  <a href="SenCommittees.asp?GA=98&amp;MemberID=1911">
   Committees
  </a>
 </td>
 <td align="center" bgcolor="white" class="detail" width="15%">
  32
 </td>
 <td align="center" bgcolor="white" class="detail" width="15%">
  R
 </td>
</tr>



Vamos a descomponer esta fila en las celdas/columnas que la componen utilizando el método `select` con selectores CSS. Mirando de cerca el HTML, hay un par de maneras en que podríamos hacer esto.

* Podríamos identificar las celdas por su etiqueta `td`.
* Podríamos usar el nombre de clase `.detail`.
* Podríamos combinar ambos y utilizar el selector `td.detail`.

In [83]:
for cell in example_row.select('td'):
    print(cell)
print()

for cell in example_row.select('.detail'):
    print(cell)
print()

for cell in example_row.select('td.detail'):
    print(cell)
print()

<td bgcolor="white" class="detail" width="40%"><a class="notranslate" href="/senate/Senator.asp?GA=98&amp;MemberID=1911">Pamela J. Althoff</a></td>
<td align="center" bgcolor="white" class="detail" width="15%"><a href="SenatorBills.asp?GA=98&amp;MemberID=1911">Bills</a></td>
<td align="center" bgcolor="white" class="detail" width="15%"><a href="SenCommittees.asp?GA=98&amp;MemberID=1911">Committees</a></td>
<td align="center" bgcolor="white" class="detail" width="15%">32</td>
<td align="center" bgcolor="white" class="detail" width="15%">R</td>

<td bgcolor="white" class="detail" width="40%"><a class="notranslate" href="/senate/Senator.asp?GA=98&amp;MemberID=1911">Pamela J. Althoff</a></td>
<td align="center" bgcolor="white" class="detail" width="15%"><a href="SenatorBills.asp?GA=98&amp;MemberID=1911">Bills</a></td>
<td align="center" bgcolor="white" class="detail" width="15%"><a href="SenCommittees.asp?GA=98&amp;MemberID=1911">Committees</a></td>
<td align="center" bgcolor="white" class

Podemos confirmar que son todos iguales.

In [86]:
assert example_row.select('td') == example_row.select('.detail') == example_row.select('td.detail')

Utilicemos el selector `td.detail` para ser lo más específicos posible.

In [89]:
# Seleccionar sólo las etiquetas 'td' con clase 'detail'
detail_cells = example_row.select('td.detail')
detail_cells

[<td bgcolor="white" class="detail" width="40%"><a class="notranslate" href="/senate/Senator.asp?GA=98&amp;MemberID=1911">Pamela J. Althoff</a></td>,
 <td align="center" bgcolor="white" class="detail" width="15%"><a href="SenatorBills.asp?GA=98&amp;MemberID=1911">Bills</a></td>,
 <td align="center" bgcolor="white" class="detail" width="15%"><a href="SenCommittees.asp?GA=98&amp;MemberID=1911">Committees</a></td>,
 <td align="center" bgcolor="white" class="detail" width="15%">32</td>,
 <td align="center" bgcolor="white" class="detail" width="15%">R</td>]

La mayoría de las veces, nos interesa el **texto** real de un sitio web, no sus etiquetas. Recuerda que para obtener el texto de un elemento HTML, utilizamos el miembro `text`:

In [92]:
# Mantener sólo el texto en cada una de esas celdas
row_data = [cell.text for cell in detail_cells]

print(row_data)

['Pamela J. Althoff', 'Bills', 'Committees', '32', 'R']


¡Tiene buena pinta! Ahora sólo tenemos que utilizar nuestros conocimientos básicos de Python para obtener los elementos de esta lista que queremos. Recuerda, queremos el nombre del senador, su distrito y su partido.

In [94]:
print(row_data[0]) # Nombre
print(row_data[3]) # Distrito
print(row_data[4]) # Partido

Pamela J. Althoff
32
R


## Deshacerse de las filas basura

Vimos al principio que no todas las filas que obtuvimos corresponden realmente a un senador. Tendremos que hacer algo de limpieza antes de seguir adelante. Echa un vistazo a algunos ejemplos:

In [97]:
print('Row 0:\n', rows[0], '\n')
print('Row 1:\n', rows[1], '\n')
print('Last Row:\n', rows[-1])

Row 0:
 <tr><td colspan="5">
<span class="heading">Illinois State Senators</span>
<span class="italics">  98th  General Assembly</span><br/>
<!-- 3/2/09 temp comment out until fixed for GA specific-->
<!-- add 97th ga currently no info -->
<a href="98GA_Senate_Leadership.pdf">Leadership</a> <a href="98th_Senate_Officers.pdf">Officers</a> <a href="98GA_Senate_Seating_Chart.pdf">Senate Seating Chart</a>  <span class="content"><b>Democrats:</b> 40   <b>Republicans:</b> 19</span><br/>
</td></tr> 

Row 1:
 <tr>
<td class="header" width="45%"><a class="filetab" href="javascript:Sort('LastName','',98);" title="Sort by Senator">Senator</a></td>
<td align="center" class="header" width="15%">Bills</td>
<td align="center" class="header" width="10%">Committees</td>
<td align="center" class="header" width="15%"><a class="filetab" href="javascript:Sort('DistrictNumber','',98);" title="Sort by District">District</a></td>
<td align="center" class="header" width="15%"><a class="filetab" href="javascrip

Cuando escribimos nuestro bucle for, sólo queremos que se aplique a las filas relevantes. Así que tendremos que filtrar las filas irrelevantes. La forma de hacerlo es comparar algunas de ellas con las filas que queremos, ver en qué se diferencian y formularlo en una condicional.

Como puedes imaginar, hay muchas formas posibles de hacer esto, y dependerá del sitio web. Mostraremos algunas aquí para darte una idea de cómo hacerlo.

In [99]:
# Filas malas
print(len(rows[0]))
print(len(rows[1]))

# Filas buenas
print(len(rows[2]))
print(len(rows[3]))

1
11
5
5


Quizá las filas buenas tengan una longitud de 5. Comprobémoslo:

In [101]:
good_rows = [row for row in rows if len(row) == 5]

# Comprobemos algunas filas
print(good_rows[0], '\n')
print(good_rows[-2], '\n')
print(good_rows[-1])

<tr><td bgcolor="white" class="detail" width="40%"><a class="notranslate" href="/senate/Senator.asp?GA=98&amp;MemberID=1911">Pamela J. Althoff</a></td><td align="center" bgcolor="white" class="detail" width="15%"><a href="SenatorBills.asp?GA=98&amp;MemberID=1911">Bills</a></td><td align="center" bgcolor="white" class="detail" width="15%"><a href="SenCommittees.asp?GA=98&amp;MemberID=1911">Committees</a></td><td align="center" bgcolor="white" class="detail" width="15%">32</td><td align="center" bgcolor="white" class="detail" width="15%">R</td></tr> 

<tr><td bgcolor="white" class="detail" width="40%"><a class="notranslate" href="/senate/Senator.asp?GA=98&amp;MemberID=2035">Patricia Van Pelt</a></td><td align="center" bgcolor="white" class="detail" width="15%"><a href="SenatorBills.asp?GA=98&amp;MemberID=2035">Bills</a></td><td align="center" bgcolor="white" class="detail" width="15%"><a href="SenCommittees.asp?GA=98&amp;MemberID=2035">Committees</a></td><td align="center" bgcolor="white

Hemos encontrado una fila a pie de página en nuestra lista que nos gustaría evitar. Probemos otra cosa:

In [103]:
rows[2].select('td.detail') 

[<td bgcolor="white" class="detail" width="40%"><a class="notranslate" href="/senate/Senator.asp?GA=98&amp;MemberID=1911">Pamela J. Althoff</a></td>,
 <td align="center" bgcolor="white" class="detail" width="15%"><a href="SenatorBills.asp?GA=98&amp;MemberID=1911">Bills</a></td>,
 <td align="center" bgcolor="white" class="detail" width="15%"><a href="SenCommittees.asp?GA=98&amp;MemberID=1911">Committees</a></td>,
 <td align="center" bgcolor="white" class="detail" width="15%">32</td>,
 <td align="center" bgcolor="white" class="detail" width="15%">R</td>]

In [105]:
# Fila mala
print(rows[-1].select('td.detail'), '\n')

# Fila buena
print(rows[5].select('td.detail'), '\n')

# ¿Qué tal esto?
good_rows = [row for row in rows if row.select('td.detail')]

print("Checking rows...\n")
print(good_rows[0], '\n')
print(good_rows[-1])

[] 

[<td bgcolor="EBEBEB" class="detail" width="40%"><a class="notranslate" href="/senate/Senator.asp?GA=98&amp;MemberID=2022">Jennifer Bertino-Tarrant</a></td>, <td align="center" bgcolor="EBEBEB" class="detail" width="15%"><a href="SenatorBills.asp?GA=98&amp;MemberID=2022">Bills</a></td>, <td align="center" bgcolor="EBEBEB" class="detail" width="15%"><a href="SenCommittees.asp?GA=98&amp;MemberID=2022">Committees</a></td>, <td align="center" bgcolor="EBEBEB" class="detail" width="15%">49</td>, <td align="center" bgcolor="EBEBEB" class="detail" width="15%">D</td>] 

Checking rows...

<tr><td bgcolor="white" class="detail" width="40%"><a class="notranslate" href="/senate/Senator.asp?GA=98&amp;MemberID=1911">Pamela J. Althoff</a></td><td align="center" bgcolor="white" class="detail" width="15%"><a href="SenatorBills.asp?GA=98&amp;MemberID=1911">Bills</a></td><td align="center" bgcolor="white" class="detail" width="15%"><a href="SenCommittees.asp?GA=98&amp;MemberID=1911">Committees</a></

Parece que hemos encontrado algo que ha funcionado.

## Unir todo en un bucle

Ahora que hemos visto cómo obtener los datos que queremos de una fila, así como filtrar las filas que no queremos, vamos a ponerlo todo junto en un bucle.

In [108]:
# Definir lista de almacenamiento
members = []

# Deshazte de las filas basura
valid_rows = [row for row in rows if row.select('td.detail')]

# Recorrer todas las filas
for row in valid_rows:
    # Seleccionar sólo las etiquetas 'td' con clase 'detail
    detail_cells = row.select('td.detail')
    # Mantener sólo el texto en cada una de esas celdas
    row_data = [cell.text for cell in detail_cells]
    # Recopilar información
    name = row_data[0]
    district = int(row_data[3])
    party = row_data[4]
    # Almacenar en una tupla
    senator = (name, district, party)
    # Añadir a la lista
    members.append(senator)

In [110]:
# Debería ser 61
len(members)

61

Echemos un vistazo a lo que tenemos en `members`.

In [112]:
print(members[:5])

[('Pamela J. Althoff', 32, 'R'), ('Jason A. Barickman', 53, 'R'), ('Scott M Bennett', 52, 'D'), ('Jennifer Bertino-Tarrant', 49, 'D'), ('Daniel Biss', 9, 'D')]


## 🥊  Desafío: Obtener elementos `href` que apunten a las facturas de los afiliados  

El código anterior recupera información sobre:  

- el nombre del senador,
- su número de distrito,
- y su partido.

Ahora queremos recuperar la URL de la lista de proyectos de ley de cada senador. Cada URL seguirá un formato específico.  

El formato de la lista de proyectos de ley para un senador determinado es:

`http://www.ilga.gov/senate/SenatorBills.asp?GA=98&MemberID=[MEMBER_ID]&Primary=True`

para obtener algo como:

`http://www.ilga.gov/senate/SenatorBills.asp?MemberID=1911&GA=98&Primary=True`

en el que `MEMBER_ID=1911`. 

Debería poder ver que, desafortunadamente, `MEMBER_ID` no es actualmente algo que se extraiga en nuestro código de raspado.

Tu tarea inicial es modificar el código anterior para que también **recuperemos la URL completa que apunta a la página correspondiente de los proyectos de ley patrocinados por los partidos**, para cada miembro, y la devolvamos junto con su nombre, distrito y partido.

Consejos: 

* Para ello, querrás obtener el elemento de anclaje apropiado (`<a>`) en cada fila de legisladores de la tabla. Puedes utilizar de nuevo el método `.select()` en el objeto `row` en el bucle para hacer esto - similar al comando que encuentra todas las celdas `td.detail` en la fila. Recuerda que sólo queremos el enlace a los proyectos de ley del legislador, no a los comités ni a la página de perfil del legislador.
* El HTML de los elementos de anclaje se verá como `<a href=«/senate/Senator.asp/...»>Bills</a>`. La cadena del atributo `href` contiene el enlace **relativo** que buscamos. Puedes acceder a un atributo de un objeto `Tag` de BeatifulSoup de la misma forma que accedes a un diccionario de Python: `anchor['attributeName']`. Consulte la <a href=«http://www.crummy.com/software/BeautifulSoup/bs4/doc/#tag»>documentación</a> para obtener más detalles.
* Hay un _lot_ de maneras diferentes de utilizar BeautifulSoup para hacer las cosas. lo que usted necesita hacer para sacar el `href` está bien.

El código ha sido parcialmente rellenado para usted. Rellénalo donde dice `#TU CÓDIGO AQUÍ`. Guarda la ruta en un objeto llamado `full_path`.

In [117]:
# Realizar una solicitud GET
req = requests.get('http://www.ilga.gov/senate/default.asp?GA=98')
# Leer el contenido de la respuesta del servidor
src = req.text
# Soup it
soup = BeautifulSoup(src, "lxml")
# Crear una lista vacía para almacenar nuestros datos
members = []

# Devuelve todos los selectores css 'tr tr tr' de la página
rows = soup.select('tr tr tr')
# Deshacerse de las filas basura
rows = [row for row in rows if row.select('td.detail')]

# Recorrer todas las filas
for row in rows:
    # Seleccionar sólo las etiquetas 'td' con clase 'detail'
    detail_cells = row.select('td.detail') 
    # Conserva sólo el texto de cada una de esas celdas
    row_data = [cell.text for cell in detail_cells]
    # Recopilar información
    name = row_data[0]
    district = int(row_data[3])
    party = row_data[4]

    # TU CÓDIGO AQUÍ
    # Extraer href
    href = row.select('a')[1]['href']
    # Crear ruta completa
    full_path = "http://www.ilga.gov/senate/" + href + "&Primary=True"

    # Almacenar en una tupla
    senator = (name, district, party, full_path)
    # Añadir a la lista
    members.append(senator)

In [119]:
members[:5]

[('Pamela J. Althoff',
  32,
  'R',
  'http://www.ilga.gov/senate/SenatorBills.asp?GA=98&MemberID=1911&Primary=True'),
 ('Jason A. Barickman',
  53,
  'R',
  'http://www.ilga.gov/senate/SenatorBills.asp?GA=98&MemberID=2018&Primary=True'),
 ('Scott M Bennett',
  52,
  'D',
  'http://www.ilga.gov/senate/SenatorBills.asp?GA=98&MemberID=2272&Primary=True'),
 ('Jennifer Bertino-Tarrant',
  49,
  'D',
  'http://www.ilga.gov/senate/SenatorBills.asp?GA=98&MemberID=2022&Primary=True'),
 ('Daniel Biss',
  9,
  'D',
  'http://www.ilga.gov/senate/SenatorBills.asp?GA=98&MemberID=2020&Primary=True')]

## 🥊  Desafío: Modularice su código

Convierta el código anterior en una función que acepte una URL, rastree la URL en busca de sus senadores y devuelva una lista de tuplas con información sobre cada senador. 

In [121]:
# TU CÓDIGO AQUÍ
def get_members(url):
    # Hacer una petición GET
    req = requests.get(url)
    # Leer el contenido de la respuesta del servidor
    src = req.text
    # Soup it
    soup = BeautifulSoup(src, "lxml")
    # Crear una lista vacía para almacenar nuestros datos
    members = []

    # Devuelve todos los selectores css 'tr tr tr' de la página
    rows = soup.select('tr tr tr')
    # Deshacerse de las filas basura
    rows = [row for row in rows if row.select('td.detail')]

    # Recorrer todas las filas
    for row in rows:
        # Seleccionar sólo las etiquetas 'td' con clase 'detail
        detail_cells = row.select('td.detail') 
        # Mantener sólo el texto en cada una de esas celdas
        row_data = [cell.text for cell in detail_cells]
        # Recoger información
        name = row_data[0]
        district = int(row_data[3])
        party = row_data[4]

        # Extraer href
        href = row.select('a')[1]['href']
        # Crear ruta completa
        full_path = "http://www.ilga.gov/senate/" + href + "&Primary=True"

        # Almacenar en una tupla
        senator = (name, district, party, full_path)
        # Añadir a la lista
        members.append(senator)
    return(members)


In [123]:
# Probar el código
url = 'http://www.ilga.gov/senate/default.asp?GA=98'
senate_members = get_members(url)
len(senate_members)

61

## 🥊 Reto para llevar a casa: Escribir una función rascadora

Queremos raspar las páginas web correspondientes a los proyectos de ley patrocinados por cada uno de ellos.

Escribe una función llamada `get_bills(url)` para analizar una URL de proyecto de ley dada. Esto implicará:

  - solicitando la URL mediante la biblioteca <a href="http://docs.python-requests.org/en/latest/">`requests`</a>.
  - utilizando las características de la biblioteca `BeautifulSoup` para encontrar todos los elementos `<td>` con la clase `billlist`.
  - devuelve una _lista_ de tuplas, cada una con:
      - descripción (2ª columna)
      - cámara (S o H) (3ª columna)
      - la última acción (4ª columna)
      - fecha de la última acción (5ª columna)
      
Esta función se ha completado parcialmente. Rellene el resto.

In [132]:
def get_bills(url):
    src = requests.get(url).text
    soup = BeautifulSoup(src)
    rows = soup.select('tr')
    bills = []
    for row in rows:
        # Obtener todas las celdas de la lista de facturas
        cells = row.select('td.billlist')
        # Hay que tener en cuenta que el nombre del senador no es una clase de lista de proyectos de ley.
        if len(cells) == 5:
            row_text = [cell.text for cell in cells]
            # Extraer información del texto de las filas
            bill_id = row_text[0]
            description = row_text[1]
            chamber = row_text[2]
            last_action = row_text[3]
            last_action_date = row_text[4]
            # Consolidar información sobre proyecto de ley
            bill = (bill_id, description, chamber, last_action, last_action_date)
            bills.append(bill)
    return bills

In [134]:
# Probando código
test_url = senate_members[0][3]
get_bills(test_url)[0:5]

[('SB27', 'MEDICAID BUDGET NOTE ACT', 'S', 'Session Sine Die', '1/13/2015'),
 ('SB28',
  'HOMELESS VETERANS SHELTER ACT',
  'S',
  'Session Sine Die',
  '1/13/2015'),
 ('SB29', 'ROAD FUND-NO TRANSFERS', 'S', 'Session Sine Die', '1/13/2015'),
 ('SB33',
  'EPA-RULES-DOCUMENT SUBMISSION',
  'S',
  'Public Act . . . . . . . . . 98-0072',
  '7/15/2013'),
 ('SB104',
  'MIN WAGE-OVERTIME-ALTERN SHIFT',
  'S',
  'Session Sine Die',
  '1/13/2015')]

### Rechazar todos los proyectos de ley

Por último, crea un diccionario `bills_dict` que asigne un número de distrito (la clave) a una lista de proyectos de ley (el valor) procedentes de ese distrito. Puedes hacerlo recorriendo todos los miembros del senado en `members_dict` y llamando a `get_bills()` para cada una de sus URLs de proyectos de ley asociadas.

**NOTA:** por favor llama a la función `time.sleep(1)` en cada iteración del bucle, para que no destruyamos la web del estado.

In [137]:
bills_dict = {}
for member in senate_members[:5]:
    bills_dict[member[1]] = get_bills(member[3])
    time.sleep(1)

In [138]:
bills_dict[52]

[('SR1730',
  'MEMORIAL - THOMAS G. HAYS',
  'S',
  'Resolution Adopted',
  '1/13/2015'),
 ('SR1730',
  'MEMORIAL - THOMAS G. HAYS',
  'S',
  'Resolution Adopted',
  '1/13/2015'),
 ('SR1730',
  'MEMORIAL - THOMAS G. HAYS',
  'S',
  'Resolution Adopted',
  '1/13/2015')]